# T1552.007 - Kubernetes List Secrets
Adversaries may gather credentials via APIs within a containers environment. APIs in these environments, such as the Docker API and Kubernetes APIs, allow a user to remotely manage their container resources and cluster components.(Citation: Docker API)(Citation: Kubernetes API)

An adversary may access the Docker API to collect logs that contain credentials to cloud, container, and various other resources in the environment.(Citation: Unit 42 Unsecured Docker Daemons) An adversary with sufficient permissions, such as via a pod's service account, may also use the Kubernetes API to retrieve credentials from the Kubernetes API server. These credentials may include those needed for Docker API authentication or secrets from Kubernetes cluster components. 

## Atomic Tests

### Atomic Test #1 - List All Secrets
A Kubernetes secret is an object that lets users store and manage sensitive information, such as passwords and connection strings in the cluster. Secrets can be consumed by reference in the pod configuration. Attackers who have permissions to retrieve the secrets from the API server (by using the pod service account, for example) can access sensitive information that might include credentials to various services or provide further access to the cluster.
[More information about secrets](https://kubernetes.io/docs/concepts/configuration/secret/).

This test will make a request to the Kubernetes api at the `/api/v1/secrets` endpoint requesting every secret stored within the cluster.

**Supported Platforms:** containers
#### Dependencies:  Run with `None`!
##### Description: kubectl must be installed

##### Check Prereq Commands:
```None
which kubectl

```
##### Get Prereq Commands:
```None
echo "kubectl not installed, please install kubectl (https://kubernetes.io/docs/tasks/tools/)"

```

In [ ]:
Invoke-AtomicTest T1552.007 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
kubectl get secrets --all-namespaces
```

In [ ]:
Invoke-AtomicTest T1552.007 -TestNumbers 1

### Atomic Test #2 - ListSecrets
A Kubernetes secret is an object that lets users store and manage sensitive information, such as passwords and connection strings in the cluster. Secrets can be consumed by reference in the pod configuration. Attackers who have permissions to retrieve the secrets from the API server (by using the pod service account, for example) can access sensitive information that might include credentials to various services.

**Supported Platforms:** containers
#### Dependencies:  Run with `None`!
##### Description: kubectl must be installed

##### Check Prereq Commands:
```None
which kubectl

```
##### Get Prereq Commands:
```None
echo "kubectl must be installed manually"

```

In [ ]:
Invoke-AtomicTest T1552.007 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
kubectl get secrets -n default
```

In [ ]:
Invoke-AtomicTest T1552.007 -TestNumbers 2

### Atomic Test #3 - Cat the contents of a Kubernetes service account token file
Access the Kubernetes service account access token stored within a container in a cluster.

**Supported Platforms:** linux
#### Dependencies:  Run with `sh`!
##### Description: Verify docker is installed.
##### Check Prereq Commands:
```sh
which docker

```
##### Get Prereq Commands:
```sh
if [ "" == "`which docker`" ]; then echo "Docker Not Found"; if [ -n "`which apt-get`" ]; then sudo apt-get -y install docker ; elif [ -n "`which yum`" ]; then sudo yum -y install docker ; fi ; else echo "Docker installed"; fi

```
##### Description: Verify docker service is running.
##### Check Prereq Commands:
```sh
sudo systemctl status docker

```
##### Get Prereq Commands:
```sh
sudo systemctl start docker

```
##### Description: Verify kind is in the path.
##### Check Prereq Commands:
```sh
which kind

```
##### Get Prereq Commands:
```sh
curl -Lo ./kind https://kind.sigs.k8s.io/dl/v0.10.0/kind-linux-amd64
chmod +x ./kind
mv kind /usr/bin/kind

```
##### Description: Verify kind-atomic-cluster is created
##### Check Prereq Commands:
```sh
sudo kind get clusters

```
##### Get Prereq Commands:
```sh
sudo kind create cluster --name atomic-cluster

```
##### Description: Verify kubectl is in path
##### Check Prereq Commands:
```sh
which kubectl

```
##### Get Prereq Commands:
```sh
curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
chmod +x ./kubectl
mv kubectl /usr/bin/kubectl

```
##### Description: Verify atomic-pod is running.
##### Check Prereq Commands:
```sh
kubectl --context kind-atomic-cluster get pods |grep atomic-pod

```
##### Get Prereq Commands:
```sh
kubectl --context kind-atomic-cluster run atomic-pod --image=alpine --command -- sleep infinity

```

In [ ]:
Invoke-AtomicTest T1552.007 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
kubectl --context kind-atomic-cluster exec atomic-pod -- cat /run/secrets/kubernetes.io/serviceaccount/token
```

In [ ]:
Invoke-AtomicTest T1552.007 -TestNumbers 3

## Detection
Establish centralized logging for the activity of container and Kubernetes cluster components. Monitor logs for actions that could be taken to gather credentials to container and cloud infrastructure, including the use of discovery API calls by new or unexpected users and APIs that access Docker logs.

It may be possible to detect adversary use of credentials they have obtained such as in [Valid Accounts](https://attack.mitre.org/techniques/T1078).